In [49]:
import pandas as pd
import sqlite3

## create a connection to the database using the library sqlite3

In [50]:
file_path = '../data/checking-logs.sqlite'
con = sqlite3.connect(file_path)
cur = con.cursor()

## get the schema of the table test

In [51]:
query = 'PRAGMA table_info(test);'
pd.read_sql(query, con)

,cid,name,type,notnull,dflt_value,pk
0,0,uid,TEXT,0,None,0
1,1,labname,TEXT,0,None,0
2,2,first_commit_ts,TIMESTAMP,0,None,0
3,3,first_view_ts,TIMESTAMP,0,None,0


## get only the first 10 rows of the table test to check what the table looks like

In [52]:
query = 'SELECT * FROM test LIMIT 10'
pd.read_sql(query, con)

,uid,labname,first_commit_ts,first_view_ts
0,user_1,laba04,2020-04-26 17:06:18.462708,2020-04-26 21:53:59.624136
1,user_1,laba04s,2020-04-26 17:12:11.843671,2020-04-26 21:53:59.624136
2,user_1,laba05,2020-05-02 19:15:18.540185,2020-04-26 21:53:59.624136
3,user_1,laba06,2020-05-17 16:26:35.268534,2020-04-26 21:53:59.624136
4,user_1,laba06s,2020-05-20 12:23:37.289724,2020-04-26 21:53:59.624136
5,user_1,project1,2020-05-14 20:56:08.898880,2020-04-26 21:53:59.624136
6,user_10,laba04,2020-04-25 08:24:52.696624,2020-04-18 12:19:50.182714
7,user_10,laba04s,2020-04-25 08:37:54.604222,2020-04-18 12:19:50.182714
8,user_10,laba05,2020-05-01 19:27:26.063245,2020-04-18 12:19:50.182714
9,user_10,laba06,2020-05-19 11:39:28.885637,2020-04-18 12:19:50.182714


## find among all the users the minimum value of the delta between the first commit of the user and the deadline of the corresponding lab using only one query


In [53]:
query = 'SELECT uid,' \
        '       min(cast((JulianDay(test.first_commit_ts) - (JulianDay(deadlines.deadlines, "unixepoch"))) * 24 as INTEGER)) AS min ' \
        'FROM test ' \
        'LEFT JOIN deadlines ' \
        'ON test.labname = deadlines.labs ' \
        'WHERE labname != "project1"'
df_min = pd.read_sql(query, con)
df_min

,uid,min
0,user_30,-202


## do the same thing, but for the maximum, using only one query, the dataframe name is df_max

In [54]:
query = 'SELECT uid,' \
        '       max(cast((JulianDay(test.first_commit_ts) - (JulianDay(deadlines.deadlines, "unixepoch"))) * 24 as INTEGER)) AS max ' \
        'FROM test ' \
        'LEFT JOIN deadlines ' \
        'ON test.labname = deadlines.labs ' \
        'WHERE labname != "project1"'
df_max = pd.read_sql(query, con)
df_max

,uid,max
0,user_25,-2


## o the same thing but for the average, using only one query, this time your dataframe should not include the uid column, and the dataframe name is df_avg

In [55]:
query = 'SELECT avg(cast((JulianDay(test.first_commit_ts) - (JulianDay(deadlines.deadlines, "unixepoch"))) * 24 as INTEGER)) AS avg ' \
        'FROM test ' \
        'LEFT JOIN deadlines ' \
        'ON test.labname = deadlines.labs ' \
        'WHERE labname != "project1"'
df_avg = pd.read_sql(query, con)
df_avg

,avg
0,-89.125


## we want to test the hypothesis that the users who visited the newsfeed just a few times have the lower delta between the first commit and the deadline. To do this, you need to calculate the correlation coefficient between the number of pageviews and the difference

In [56]:
query = 'WITH tmp AS (SELECT test.uid,' \
        '                    avg(cast((JulianDay(test.first_commit_ts) - (JulianDay(deadlines.deadlines, "unixepoch"))) * 24 as INTEGER)) AS avg_diff' \
        '             FROM test ' \
        '             JOIN deadlines ' \
        '             ON test.labname = deadlines.labs ' \
        '             GROUP BY test.uid) ' \
        'SELECT tmp.uid, tmp.avg_diff, count(pageviews.datetime) AS pageviews ' \
        'FROM tmp ' \
        'JOIN pageviews ' \
        'ON tmp.uid = pageviews.uid ' \
        'GROUP BY tmp.uid'
views_diff = pd.read_sql(query, con)
views_diff

,uid,avg_diff,pageviews
0,user_1,-62.166667,28
1,user_10,-78.833333,89
2,user_14,-185.750000,143
3,user_17,-166.000000,47
4,user_18,-37.500000,3
5,user_19,-91.200000,16
6,user_21,-106.800000,10
7,user_25,-111.166667,179
8,user_28,-89.500000,149
9,user_3,-124.166667,317


In [57]:
views_diff.corr()

,avg_diff,pageviews
avg_diff,1.000000,-0.069302
pageviews,-0.069302,1.000000


## close the connection

In [58]:
con.close()